<a href="https://colab.research.google.com/github/komo135/tradingrl/blob/master/dqac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My Drive
%load_ext Cython

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [0]:
!pip install ta

In [0]:
import numpy as np
from scipy.special import expit
import pandas as pd
import pickle
from numba import jit as njit
from functools import lru_cache
from multiprocessing import Pool
import os
import time
import random
import ta
from net import *
from memory import *
from reward import *
import traceback
from IPython.display import clear_output

In [0]:
def swish(x):
    x *= tf.nn.sigmoid(x)
    return x

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def dqn(x):
  with tf.variable_scope("dqn", reuse=False):
    x1 = tf.keras.layers.Conv1D(128,8,1,padding="causal",activation=tf.nn.relu)(x)
    x2 = tf.keras.layers.Conv1D(128,4,1,padding="causal",activation=tf.nn.relu)(x)
    x3 = tf.keras.layers.Conv1D(128,2,1,padding="causal",activation=tf.nn.relu)(x)
    x = tf.keras.layers.Concatenate()([x1,x2,x3])
    x = tf.keras.layers.Conv1D(128,3,1,padding="valid",activation=tf.nn.relu)(x)
    f = tf.keras.layers.Flatten()(x)
    out = tf.keras.layers.Dense(128,tf.nn.relu)(f)
    out = tf.keras.layers.Dense(2)(out)
    return out

def actor(x,dqn):
  with tf.variable_scope("actor", reuse=False):
    inputs = tf.keras.layers.Flatten()(x)
    x1 = tf.keras.layers.Conv1D(128,8,1,padding="causal",activation=tf.nn.relu)(x)
    x2 = tf.keras.layers.Conv1D(128,4,1,padding="causal",activation=tf.nn.relu)(x)
    x3 = tf.keras.layers.Conv1D(128,2,1,padding="causal",activation=tf.nn.relu)(x)
    x = tf.keras.layers.Concatenate()([x1,x2,x3])
    x = tf.keras.layers.Conv1D(128,3,1,padding="valid",activation=tf.nn.relu)(x)
    f = tf.keras.layers.Flatten()(x)
    out = tf.keras.layers.Dense(128,tf.nn.relu)(f)
    out = tf.keras.layers.Dense(2)(out)
    out = out + dqn
    out = tf.keras.layers.Dense(128,tf.nn.relu)(out)
    shape = int(out.shape[-1])
    a = tf.keras.layers.Dense(shape,tf.nn.softmax)(out)
    mul = tf.keras.layers.Multiply()([out,a])
    out = tf.keras.layers.Dense(128,tf.nn.relu)(mul)
    out = tf.keras.layers.Dense(2,tf.tanh)(out)
    return out

def net(x,layer):
  for i in layer:
    x = i(x)
  return x

def critic(x,action,policy,q):
  with tf.variable_scope("critic", reuse=False):
    x1 = tf.keras.layers.Conv1D(128,8,1,padding="causal",activation=tf.nn.relu)(x)
    x2 = tf.keras.layers.Conv1D(128,4,1,padding="causal",activation=tf.nn.relu)(x)
    x3 = tf.keras.layers.Conv1D(128,2,1,padding="causal",activation=tf.nn.relu)(x)
    x = tf.keras.layers.Concatenate()([x1,x2,x3])
    x = tf.keras.layers.Conv1D(128,3,1,padding="valid",activation=tf.nn.relu)(x)
    f = tf.keras.layers.Flatten()(x)
    # x = tf.keras.layers.Dense(128,tf.nn.relu)(f)
    x1 = tf.keras.layers.Concatenate()([f,action,q])
    x2 = tf.keras.layers.Concatenate()([f,policy,q])
    layer = [tf.keras.layers.Dense(128,tf.nn.relu),tf.keras.layers.Dense(1)]
    
    x1 = net(x1,layer)
    x2 = net(x2,layer)

    return x1,x2

In [0]:
class Agent:
    def __init__(self,path,window_siz):
      self.path = path
      self.window_size = window_size
      self.STEP_SIZE = 20 * 12 * 2
      self.preproc()
      self.state_size = (None,self.window_size,self.df.shape[-1])
      print(self.state_size)
      self.memory = Memory(self.STEP_SIZE * 1000 * 5)
      self.rewards = reward3
      self.noise = 0.5
      self.noise_min = 0.1
      self.epsilon = 1.
      self.DECAY_RATE = 0.005
      self.min_epsilon = 0.1
      self.sess = tf.Session()

      with tf.variable_scope("input"):
        self.state = tf.placeholder(tf.float32, self.state_size)
        self.new_state = tf.placeholder(tf.float32, self.state_size)
        self.action = tf.placeholder(tf.float32,(None,2))
        self.reward1 = tf.placeholder(tf.float32,(None,1))
        self.reward2 = tf.placeholder(tf.float32,(None,2))

      with tf.variable_scope("model", reuse=False):
        self.dqn = dqn(self.state)
        self.policy = actor(self.state,self.dqn)
        q,q_pi = critic(self.state,self.action,self.policy,self.dqn)
      
      with tf.variable_scope("target", reuse=False):
        self.target_dqn = dqn(self.new_state)

      with tf.variable_scope("loss"):
        self.q_pi = q_pi
        self.loss = loss = (0.5 * tf.reduce_mean((self.reward1 - q) ** 2))
        dqn_loss = 0.5 * tf.reduce_mean((self.reward2 - self.dqn) ** 2)
        self.ploss = policy_loss = -tf.reduce_mean(q_pi)
        self.absolute_errors = tf.abs(self.reward1 - q)

      self.actor_opt = tf.train.AdamOptimizer(1e-3).minimize(policy_loss, var_list=get_vars('model/actor'))
      self.critic_opt = tf.train.AdamOptimizer(1e-3).minimize(loss, var_list=get_vars('model/critic'))
      self.dqn_opt = tf.train.AdamOptimizer(1e-3).minimize(dqn_loss, var_list=get_vars('model/dqn'))

      self.target_update = tf.group([tf.assign(v_targ, 0.001*v_targ + (1-0.001)*v_main)
                                for v_main, v_targ in zip(get_vars('model/dqn'), get_vars('target/dqn'))])

      target_init = tf.group([tf.assign(v_targ, v_main)
                              for v_main, v_targ in zip(get_vars('model/dqn'), get_vars('target/dqn'))])

      self.sess.run(tf.global_variables_initializer())
      self.sess.run(target_init)

    def preproc(self):
        self.dat = df = pd.read_csv(self.path)
        s = np.asanyarray(ta.stoch(df["High"],df["Low"],df["Close"],14)).reshape((-1, 1)) - np.asanyarray(ta.stoch_signal(df["High"],df["Low"],df["Close"],14)).reshape((-1, 1))
        x = np.asanyarray(ta.daily_return(df["Close"])).reshape((-1,1))
        m = np.asanyarray(ta.macd_diff(df["Close"])).reshape((-1,1))
        cross1 = np.asanyarray(ta.ema(self.dat["Close"],12)).reshape((-1, 1)) - np.asanyarray(ta.ema(self.dat["Close"],5)).reshape((-1, 1))
        trend = np.asanyarray(df[["Close"]]) - np.asanyarray(ta.ema(self.dat["Close"],50)).reshape((-1, 1))
        # x = s
        x = np.concatenate([s,x,cross1,trend,m], 1)
        y = np.asanyarray(self.dat[["Open"]])

        gen = tf.keras.preprocessing.sequence.TimeseriesGenerator(x, y, self.window_size)
        self.x = []
        self.y = []
        for i in gen:
            self.x.extend(i[0].tolist())
            self.y.extend(i[1].tolist())
        self.x = np.asanyarray(self.x)[100:] #.reshape((-1, self.window_size, x.shape[-1]))
        self.y = np.asanyarray(self.y)[100:]

        self.df = self.x[-self.STEP_SIZE:]
        self.trend = self.y[-self.STEP_SIZE:]

    def _construct_memories_and_train(self,i, replay=None):
      try:
          tree_idx, replay = self.memory.sample(128)
      except:
          self.memory = Memory(5000)

      states = np.array([a[0][0] for a in replay])
      new_states = np.array([a[0][3] for a in replay])
      actions = np.array([a[0][1] for a in replay]).reshape((-1, 2))
      act = np.array([a[0][-2] for a in replay]).reshape((-1, 1))
      rewards = np.array([a[0][2] for a in replay]).reshape((-1, 1))
      r = rewards.copy()
      done = np.array([a[0][-1] for a in replay]).reshape((-1, 1))

      q = self.sess.run(self.dqn,feed_dict={self.state:states,self.action:actions})
      new_q = self.sess.run(self.dqn,feed_dict={self.state:new_states})
      new_target_q = self.sess.run(self.target_dqn, feed_dict={self.new_state:new_states})

      for I in range(128):
        q[I,act[I]] = r[I] + done[I]# * 0.99 * new_target_q[I,np.argmax(new_q[I])]
        rewards[I] = rewards[I] + done[I] * 0.99 * new_target_q[I,np.argmax(new_q[I])]

      step_ops = [self.absolute_errors,self.critic_opt,self.dqn_opt]
      for _ in range(1):
        absolute_errors,_,_ = self.sess.run(step_ops, feed_dict={self.state: states, self.reward1:rewards ,self.reward2:q ,self.action:actions})
      if i > 20:
        if (i+1) % 2 == 0:
              loss,_ = self.sess.run([self.loss,self.actor_opt], feed_dict={self.state: states,self.reward1:rewards, self.action:actions})
              # print(loss)
          # print([rewards,loss])
      self.sess.run(self.target_update)
      self.memory.batch_update(tree_idx, absolute_errors)

    def prob(self,history):
        prob = np.asanyarray(history)
        a = np.mean(prob == 0)
        b = np.mean(prob == 1)
        c = 1 - (a + b)
        prob = [a,b,c]
        return prob

    def discount_rewards(self, r, running_add):
        running_add = running_add * 0.99 + r
        return running_add
        
    def nstep(self,r):
        running_add = 0.0
        for t in range(len(r)):
            running_add += 0.99 * r[t]

        return running_add

    def _select_action(self, state, i):
      prediction,q = self.sess.run([self.policy,self.dqn], feed_dict={self.state: [state]})
      prediction,q = prediction[0],q[0]
      q = np.abs(q) / np.sum(q)
      if i < 20:
        self.noise = 1.
      elif i == 20:
        self.noise = 0.5

      if self.noise > self.noise_min:
        self.noise *= 0.9999
      noise = 0. if (i + 1) % 5 == 0 else self.noise
      # print(prediction)
      prediction += noise * np.random.randn(2)
      q += noise * np.random.randn(2)
      q += prediction
      prediction = np.clip(prediction, -1, 1)
      noise = 0. if (i + 1) % 5 == 0 else 0.1
      q = (np.abs(q) / np.sum(q)) + noise * np.random.randn(2)
      # prediction = q
      action = np.argmax(q)
      self.pred = prediction

      return action

    def run(self, spread=10, pip_cost=1000, los_cut=600, day_pip=20, n=10,step=100):
        spread = spread / pip_cost
        self.rand = np.random.RandomState()
        lc = los_cut / pip_cost
        # h = self.rand.randint(self.x.shape[0]-(self.STEP_SIZE+1))
        # self.df = self.x[h:h+self.STEP_SIZE]
        # self.trend = self.y[h:h+self.STEP_SIZE]
        for i in range(100000):
            # if i % 1000 == 0:
            #   h = self.rand.randint(self.x.shape[0]-(self.STEP_SIZE+1))
            #   self.df = self.x[h:h+self.STEP_SIZE]
            #   self.trend = self.y[h:h+self.STEP_SIZE]
            done = 1.0
            position = 3
            pip = []
            provisional_pip = []
            running_add = 0.0
            total_pip = 0.0
            old_reword = 0.0
            states = []
            h_a = []
            h_a1 = []
            h_r = []
            h_p = []
            old = np.asanyarray(0)
            self.history = []
            for t in  range(0, len(self.trend)-1):
                action = self._select_action(self.df[t],i)
                h_a.append(self.pred)
                h_a1.append(action)
                self.history.append(action)
                
                states,provisional_pip,position,total_pip = self.rewards(self.trend[t],pip,provisional_pip,
                                    action,position,states,pip_cost,spread,total_pip,lc=min([lc/2,100/pip_cost]))
                h_p.append(position)
                reward =  (total_pip - old_reword) * 100
                old_reword = total_pip
                h_r.append(reward)

                # running_add = self.discount_rewards(reward,running_add)
                # r_d = int(running_add * (pip_cost / 100)) * 100
                # # r_d = int(np.mean(np.array(provisional_pip) > 0)*100) if len(provisional_pip) != 0 else 0.
                # if t == len(self.trend)-5:
                #     done = 0.
                # exp = self.df[t], h_a[t], r_d, self.df[t+1], done
                # self.memory.store(exp)
            self.epsilon = self.min_epsilon + (1.0 - self.min_epsilon) * np.exp(-self.DECAY_RATE * i)
            for t in range(0, len(self.trend)-1):
                tau = t - n + 1
                if tau >= 0:
                    rewards = self.nstep(h_r[tau+1:tau+n])
                    exp = self.df[tau], h_a[tau], int(rewards), self.df[tau+n], h_a1[tau], done
                    self.memory.store(exp)

            try:
              self._construct_memories_and_train(i)
            except:
              traceback.print_exc()

            if i % 1000 == 0:
              clear_output()

            if (i + 1) % 5 == 0:
                # loss = np.mean(ae)
                self.pip = np.asanyarray(provisional_pip) * pip_cost
                self.pip = [p if p >= -los_cut else -los_cut for p in self.pip]
                self.total_pip = np.sum(self.pip)
                mean_pip = self.total_pip / (t + 1)
                trade_accuracy = np.mean(np.asanyarray(self.pip) > 0)
                self.trade = trade_accuracy
                mean_pip *= day_pip
                prob = self.prob(self.history)
                position_prob = self.prob(h_p)

                # print("loss =", loss)
                # print("")
                print('action probability = ', prob)
                print("buy = ", position_prob[1], " sell = ", position_prob[-1])
                print('trade accuracy = ', trade_accuracy)
                print('epoch: %d, total rewards: %f, mean rewards: %f' % (i + 1, float(self.total_pip), float(mean_pip)))
                print("")

In [0]:
window_size = 30
path = "audpred1440.csv"
agent = Agent(path,window_size)

agent.run()

action probability =  [0.42379958246346555, 0.5762004175365344, 0.0]
buy =  0.42379958246346555  sell =  0.5762004175365345
trade accuracy =  0.9300847457627118
epoch: 8005, total rewards: 1213119.000000, mean rewards: 50652.150313

action probability =  [0.42171189979123175, 0.5782881002087683, 0.0]
buy =  0.42171189979123175  sell =  0.5782881002087683
trade accuracy =  0.9426751592356688
epoch: 8010, total rewards: 1214415.000000, mean rewards: 50706.263048

action probability =  [0.4363256784968685, 0.5636743215031316, 0.0]
buy =  0.4363256784968685  sell =  0.5636743215031315
trade accuracy =  0.9531914893617022
epoch: 8015, total rewards: 1226078.000000, mean rewards: 51193.235908

action probability =  [0.4551148225469729, 0.5448851774530271, 0.0]
buy =  0.4551148225469729  sell =  0.5448851774530271
trade accuracy =  0.9553191489361702
epoch: 8020, total rewards: 1226709.000000, mean rewards: 51219.582463

action probability =  [0.4592901878914405, 0.5407098121085595, 0.0]
buy 

In [0]:
    def _select_action(self, state, i): 
      prediction,q = self.sess.run([self.policy,self.dqn], feed_dict={self.state: [state]})
      prediction,q = prediction[0],q[0]

      q = np.abs(q) / np.sum(q)
      noise = 0.2 if (i+1) % 5 != 0 else 0.
      prediction += 0.2 * np.random.randn(2)
      q += 0.2 * np.random.randn(2)
      prediction = np.clip(prediction,-1,1)
      q = np.clip(q,-1,1)
      if np.random.rand() < self.epsilon and (i+1) % 5 != 0:
        q += prediction
        q = softmax(q)
        exp = np.exp(q)
        exp /= sum(exp)
        action = np.random.choice(range(2), p=exp)
      else:
        q += prediction
        action = np.argmax(q)
      self.pred = prediction
      return action

In [0]:
agent.pred